In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [32]:
import findspark 
findspark.init()
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark ML example on titanic data").getOrCreate()


In [33]:
import glob
file_list = glob.glob('./data_japan_house/*')

In [34]:
path_and_file = './data_japan_house/13_Tokyo_20053_20182.csv'
sdf = spark.read.csv(path_and_file,header='True',inferSchema='True',encoding='SHIFT-JIS')
sdf.show(1)

+---+---------+---+-------+-----+-----+---+------+---------+---------+----+----+------+----------+-----+---+--------+-----+-----+---------+-------+-------+-------+----------+----+-------+------+----------+----+------+
| No|       種類| 地域|市区町村コード|都道府県名|市区町村名|地区名|最寄駅：名称|最寄駅：距離（分）| 取引価格（総額）| 坪単価| 間取り|面積（�u）|取引価格（�u単価）|土地の形状| 間口|延床面積（�u）|  建築年|建物の構造|       用途|今後の利用目的|前面道路：方位|前面道路：種類|前面道路：幅員（ｍ）|都市計画|建ぺい率（％）|容積率（％）|      取引時点|  改装|取引の事情等|
+---+---------+---+-------+-----+-----+---+------+---------+---------+----+----+------+----------+-----+---+--------+-----+-----+---------+-------+-------+-------+----------+----+-------+------+----------+----+------+
|  1|宅地(土地と建物)|商業地|  13101|  東京都| 千代田区|飯田橋|   飯田橋|        3|250000000|null|null|    80|      null| ほぼ台形|6.8|     330|昭和61年|   ＲＣ|住宅、事務所、店舗|    事務所|     南西|     区道|       8.0|商業地域|     80|   500|平成30年第２四半期|null|  null|
+---+---------+---+-------+-----+-----+---+------+---------+---------+----+----+------+----------+-----+---+--------+-----+-----

In [35]:
sdf.printSchema()

root
 |-- No: integer (nullable = true)
 |-- 種類: string (nullable = true)
 |-- 地域: string (nullable = true)
 |-- 市区町村コード: integer (nullable = true)
 |-- 都道府県名: string (nullable = true)
 |-- 市区町村名: string (nullable = true)
 |-- 地区名: string (nullable = true)
 |-- 最寄駅：名称: string (nullable = true)
 |-- 最寄駅：距離（分）: string (nullable = true)
 |-- 取引価格（総額）: long (nullable = true)
 |-- 坪単価: integer (nullable = true)
 |-- 間取り: string (nullable = true)
 |-- 面積（�u）: string (nullable = true)
 |-- 取引価格（�u単価）: integer (nullable = true)
 |-- 土地の形状: string (nullable = true)
 |-- 間口: string (nullable = true)
 |-- 延床面積（�u）: string (nullable = true)
 |-- 建築年: string (nullable = true)
 |-- 建物の構造: string (nullable = true)
 |-- 用途: string (nullable = true)
 |-- 今後の利用目的: string (nullable = true)
 |-- 前面道路：方位: string (nullable = true)
 |-- 前面道路：種類: string (nullable = true)
 |-- 前面道路：幅員（ｍ）: double (nullable = true)
 |-- 都市計画: string (nullable = true)
 |-- 建ぺい率（％）: integer (nullable = true)
 |-- 容積率（％）: integer (

In [36]:
from pyspark.sql.functions import col

In [37]:
sdf.show(1)

+---+---------+---+-------+-----+-----+---+------+---------+---------+----+----+------+----------+-----+---+--------+-----+-----+---------+-------+-------+-------+----------+----+-------+------+----------+----+------+
| No|       種類| 地域|市区町村コード|都道府県名|市区町村名|地区名|最寄駅：名称|最寄駅：距離（分）| 取引価格（総額）| 坪単価| 間取り|面積（�u）|取引価格（�u単価）|土地の形状| 間口|延床面積（�u）|  建築年|建物の構造|       用途|今後の利用目的|前面道路：方位|前面道路：種類|前面道路：幅員（ｍ）|都市計画|建ぺい率（％）|容積率（％）|      取引時点|  改装|取引の事情等|
+---+---------+---+-------+-----+-----+---+------+---------+---------+----+----+------+----------+-----+---+--------+-----+-----+---------+-------+-------+-------+----------+----+-------+------+----------+----+------+
|  1|宅地(土地と建物)|商業地|  13101|  東京都| 千代田区|飯田橋|   飯田橋|        3|250000000|null|null|    80|      null| ほぼ台形|6.8|     330|昭和61年|   ＲＣ|住宅、事務所、店舗|    事務所|     南西|     区道|       8.0|商業地域|     80|   500|平成30年第２四半期|null|  null|
+---+---------+---+-------+-----+-----+---+------+---------+---------+----+----+------+----------+-----+---+--------+-----+-----

In [38]:
pdf = pd.read_csv('./data_japan_house/13_Tokyo_20053_20182.csv',encoding="cp932")
pdf.head(5)

C:\Users\ko_kaku\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,No,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,取引価格（総額）,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
0,1,宅地(土地と建物),商業地,13101,東京都,千代田区,飯田橋,飯田橋,3,250000000,...,事務所,南西,区道,8.0,商業地域,80.0,500.0,平成30年第２四半期,NaN,NaN
1,2,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,3,22000000,...,住宅,NaN,NaN,NaN,商業地域,80.0,700.0,平成30年第２四半期,未改装,NaN
2,3,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,1,69000000,...,住宅,NaN,NaN,NaN,商業地域,80.0,600.0,平成30年第２四半期,未改装,NaN
3,4,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,3,21000000,...,事務所,NaN,NaN,NaN,商業地域,80.0,700.0,平成30年第２四半期,未改装,NaN
4,5,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,4,45000000,...,住宅,NaN,NaN,NaN,商業地域,80.0,700.0,平成30年第１四半期,改装済,NaN


In [39]:
pdf.dtypes

No              int64
種類             object
地域             object
市区町村コード         int64
都道府県名          object
市区町村名          object
地区名            object
最寄駅：名称         object
最寄駅：距離（分）      object
取引価格（総額）        int64
坪単価           float64
間取り            object
面積（㎡）          object
取引価格（㎡単価）     float64
土地の形状          object
間口             object
延床面積（㎡）        object
建築年            object
建物の構造          object
用途             object
今後の利用目的        object
前面道路：方位        object
前面道路：種類        object
前面道路：幅員（ｍ）    float64
都市計画           object
建ぺい率（％）       float64
容積率（％）        float64
取引時点           object
改装             object
取引の事情等         object
dtype: object

In [40]:
pdf.isnull().any()

No            False
種類            False
地域             True
市区町村コード       False
都道府県名         False
市区町村名         False
地区名            True
最寄駅：名称         True
最寄駅：距離（分）      True
取引価格（総額）      False
坪単価            True
間取り            True
面積（㎡）         False
取引価格（㎡単価）      True
土地の形状          True
間口             True
延床面積（㎡）        True
建築年            True
建物の構造          True
用途             True
今後の利用目的        True
前面道路：方位        True
前面道路：種類        True
前面道路：幅員（ｍ）     True
都市計画           True
建ぺい率（％）        True
容積率（％）         True
取引時点          False
改装             True
取引の事情等         True
dtype: bool

In [41]:
pdf['地域'] = pdf.fillna('未知')

In [42]:
pdf.loc[pdf['地区名'].isnull(),'地区名'] = pdf.loc[pdf['地区名'].isnull(),'最寄駅：名称']

In [43]:
pdf.loc[pdf['最寄駅：名称'].isnull(),'最寄駅：名称'] = pdf.loc[pdf['最寄駅：名称'].isnull(),'地区名']

In [44]:
pdf.loc[pdf['地区名'].isnull()&pdf['最寄駅：名称'].isnull(),['地区名','最寄駅：名称']]

,地区名,最寄駅：名称
103566,NaN,NaN
358028,NaN,NaN
358029,NaN,NaN
367281,NaN,NaN


In [45]:
pdf = pdf.dropna(how='all',subset=['地区名','最寄駅：名称'])

In [46]:
pdf.groupby('最寄駅：距離（分）').count()['No']

最寄駅：距離（分）
0           1640
1           9181
10         19069
11         19524
12         11723
13         13985
14         12980
15         11142
16          9838
17          2360
18          7408
19          6335
1H30?2H      252
1H?1H30     1381
2          14948
20          4779
21          4157
22           814
23          3711
24          2658
25          2710
26          2076
27           296
28          1967
29          1617
2H?          143
3          22450
30分?60分    10244
4          27166
5          25950
6          30123
7          22756
8          27147
9          24629
Name: No, dtype: int64

In [47]:
li = ['1H30?2H','1H?1H30','30分?60分','2H?']
pdf = pdf.drop(pdf[pdf['最寄駅：距離（分）'].isin(li)].index)

In [48]:
mean = pdf.loc[pdf['最寄駅：距離（分）'].isnull() == False,'最寄駅：距離（分）'].astype(float).mean()
pdf[['最寄駅：距離（分）']] = pdf[['最寄駅：距離（分）']].fillna(mean)

間取りにはnullが入っている  
面積で保管したい

In [49]:
pdf = pdf.drop(pdf[pdf['面積（㎡）'].str.contains('以上')].index )

In [50]:
pdf['面積'] = pdf['面積（㎡）'].astype(int)
pdf= pdf.drop(['面積（㎡）'],axis=1)

In [51]:
features_to_fill =['間取り',
'土地の形状',
'間口','建物の構造',
'用途','今後の利用目的',
'前面道路：方位',
'前面道路：種類',
'都市計画',
'改装',
'取引の事情等']
pdf[features_to_fill] = pdf[features_to_fill].fillna('未知')

In [52]:
features_to_fill =['延床面積（㎡）',
'建築年',
'前面道路：幅員（ｍ）',  
'建ぺい率（％）' ,'容積率（％）' ]

pdf[features_to_fill]
# features_to_fill[]

,延床面積（㎡）,建築年,前面道路：幅員（ｍ）,建ぺい率（％）,容積率（％）
0,330,昭和61年,8.0,80.0,500.0
1,NaN,昭和60年,NaN,80.0,700.0
2,NaN,昭和59年,NaN,80.0,600.0
3,NaN,昭和60年,NaN,80.0,700.0
4,NaN,昭和57年,NaN,80.0,700.0
5,NaN,昭和59年,NaN,80.0,600.0
6,NaN,平成15年,NaN,80.0,500.0
7,NaN,平成24年,NaN,80.0,500.0
8,NaN,平成15年,NaN,80.0,500.0
9,330,昭和61年,8.0,80.0,500.0


In [ ]:
def year_transform():
    if re.match('昭和元年', wareki_list[i]):
        wareki_list[i] = 1926

    elif re.match('昭和',wareki_list[i]):
        wareki_list[i] = 1925 + int(re.sub(r'\D','',wareki_list[i]))

    elif re.match('平成元年', wareki_list[i]):
        wareki_list[i] = 1989

    elif re.match('平成',wareki_list[i]):
        wareki_list[i] = 1988 + int(re.sub(r'\D','',wareki_list[i]))     

    else:
        continue